# Goal
After talking a little bit with the customer, we figured out that census tracts are more useful from a ranking perspective than zip codes, and that the `tot_ratio` was a red herring. The goal for this notebook will be simple. To import the data from the source, and try to convert the information from zip codes into census tracts.

In [1]:
import pandas as pd
import os
import numpy as np

project_dir = os.path.join(os.getcwd(), os.pardir)
files = ('community_impact_scores.csv', 'opportunity_scores.csv', 'senior_scores.csv')

paths = [os.path.join(project_dir, 'data', data_file) for data_file in files]
ci_score, op_score, sr_score = [pd.read_csv(data_path) for data_path in paths]

In [2]:
del op_score['Unnamed: 3']
del op_score['Unnamed: 4']
del ci_score['Unnamed: 3']
del ci_score['Unnamed: 4']

In [3]:
sr_score.columns = ['zip', 'so', 'srb', 'shs', 'chc']
sr_score.chc = sr_score.chc.map(dict(Yes=1, No=0))

sr_score.head()

,zip,so,srb,shs,chc
0,15001,18.83%,31.05%,16.03%,1
1,15003,16.93%,33.28%,45.95%,0
2,15005,20.99%,33.19%,12.24%,0
3,15006,13.15%,48.39%,100.00%,0
4,15009,23.37%,31.80%,19.66%,0


In [4]:
ci_score['tract_id_proper'] = ci_score['CENSUS TRACT'].str.extract(r'Census Tract (.+),.+, Pennsylvania')
ci_score.drop(['CENSUS TRACT', 'COUNTY'], axis=1, inplace=True)
ci_score.columns = ['ci_score', 'tract_id_proper']
ci_score.head()

C:\Users\DT-Laptop\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,ci_score,tract_id_proper
0,2.0,301.01
1,1.0,301.02
2,3.5,302
3,2.0,303
4,2.0,304


In [5]:
op_score['AREA'] = op_score.AREA.str.extract(r'\((.+)\)')
op_score.drop(['AREA'], axis=1, inplace=True)
op_score.columns = ['zip', 'op_score']
op_score.head()

C:\Users\DT-Laptop\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,zip,op_score
0,15001,3
1,15003,3
2,15004,3
3,15005,3.5
4,15006,2.75


In [6]:
tract_zip_correlation = pd.read_excel(os.path.join(project_dir, 'data', 'ZIP_TRACT_092017.xlsx'))
tract_zip_correlation.head()

,zip,tract,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,37801,47009011603,0.035392,0.041385,0.000000,0.035849
1,37801,47009011200,0.339105,0.448480,0.333333,0.348114
2,37766,47013950800,0.086610,0.062833,0.008021,0.082289
3,37801,47009011604,0.068884,0.010135,0.133333,0.064110
4,601,72001956700,0.668537,0.420792,0.500000,0.649833


In [7]:
pa_temp = tract_zip_correlation[tract_zip_correlation.tract.astype(str).str[:2] == '42'].reset_index(drop=True)

In [8]:
pa_temp['County FIPS Code'] = pa_temp.tract.astype(str).str[2:5].astype(int)
pa_temp.drop(['res_ratio', 'bus_ratio', 'oth_ratio', 'tot_ratio'], axis=1, inplace=True)
pa_temp['tract_id'] = pa_temp.tract.astype(str).str[5:]

In [9]:
pa_temp['tract_id_proper'] = pa_temp.tract_id.astype(float)/100
pa_temp['tract_id_proper'] = pa_temp.tract_id_proper.astype(str).str.replace('\.0$', '')
pa_temp.head()

,zip,tract,County FIPS Code,tract_id,tract_id_proper
0,19438,42091207500,91,207500,2075
1,15005,42003411000,3,411000,4110
2,15001,42007604902,7,604902,6049.02
3,15009,42007602500,7,602500,6025
4,15009,42007602300,7,602300,6023


In [10]:
df = ci_score.merge(pa_temp[['zip', 'tract_id_proper']], on='tract_id_proper')
df[df['zip'] == 17550]

,ci_score,tract_id_proper,zip
13150,2.0,111,17550
13172,2.0,111,17550
13194,3.5,111,17550
13216,3.0,111,17550
13238,1.0,111,17550
13260,2.5,111,17550
13282,3.0,111,17550
13304,5.0,111,17550
13326,5.0,111,17550


In [11]:
df.tail(20)

,ci_score,tract_id_proper,zip
20809,1.5,237.21,17366
20810,1.5,237.21,17322
20811,1.5,237.21,17309
20812,1.5,237.22,17302
20813,1.5,237.22,17314
20814,1.5,237.22,17309
20815,1.5,238.21,17355
20816,1.5,238.21,17349
20817,1.5,238.21,17361
20818,2.0,238.22,17327


In [12]:
df = df.merge(op_score,on='zip').merge(sr_score, on='zip')
df.head(10)

,ci_score,tract_id_proper,zip,op_score,so,srb,shs,chc
0,2.0,301.01,17316,3.75,14.70%,23.38%,24.68%,0
1,1.0,301.02,17316,3.75,14.70%,23.38%,24.68%,0
2,2.0,308,17316,3.75,14.70%,23.38%,24.68%,0
3,1.0,308,17316,3.75,14.70%,23.38%,24.68%,0
4,3.5,308,17316,3.75,14.70%,23.38%,24.68%,0
5,2.0,301.01,17324,4,19.68%,22.42%,17.17%,0
6,3.5,302,17324,4,19.68%,22.42%,17.17%,0
7,2.0,302,17324,4,19.68%,22.42%,17.17%,0
8,3.0,302,17324,4,19.68%,22.42%,17.17%,0
9,2.0,303,17324,4,19.68%,22.42%,17.17%,0


In [13]:
df.zip = df.zip.astype('category')
dt_cols = ['op_score', 'so', 'srb', 'shs', 'chc']
df[dt_cols[1:]] = df[dt_cols[1:]].replace('%','',regex=True).astype('float')/100 #why cant I do this inplace?
df.head()

,ci_score,tract_id_proper,zip,op_score,so,srb,shs,chc
0,2.0,301.01,17316,3.75,0.147,0.2338,0.2468,0.0
1,1.0,301.02,17316,3.75,0.147,0.2338,0.2468,0.0
2,2.0,308,17316,3.75,0.147,0.2338,0.2468,0.0
3,1.0,308,17316,3.75,0.147,0.2338,0.2468,0.0
4,3.5,308,17316,3.75,0.147,0.2338,0.2468,0.0


In [14]:
df[dt_cols] = df[dt_cols].apply(pd.to_numeric, errors='coerce', axis=1)
df.dtypes

ci_score            float64
tract_id_proper      object
zip                category
op_score            float64
so                  float64
srb                 float64
shs                 float64
chc                 float64
dtype: object

In [15]:
final = df.groupby('tract_id_proper').mean().head(50)

In [16]:
final.to_csv(os.path.join(project_dir, 'data', 'joint_data_by_census.csv'))